In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
file_path = 'classifier_training_data/posts_articles_about.csv'
df = pd.read_csv(file_path)

# Display the DataFrame
df


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import string
nltk.download('wordnet')
nltk.download('stopwords')

# You may need to download WordNet data by running: nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# This line of code can be used to update the list of stopwords
# nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stopwords and lemmatize
    text = [lemmatizer.lemmatize(word) for word in text.split() if word not in stopwords]
    
    # Join the words back into one string
    text = ' '.join(text)
    
    return text

# Apply the preprocessing to every document in the data
df['content'] = df['content'].apply(preprocess_text)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the T5 dataset
class T5Dataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_text = "classify: " + row['content']
        target_text = row['label']

        inputs = self.tokenizer.encode_plus(
            input_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        targets = self.tokenizer.encode_plus(
            target_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': targets['input_ids'].flatten()
        }

tokenizer = T5Tokenizer.from_pretrained('t5-base')
train_dataset = T5Dataset(train_df, tokenizer, max_len=512)
val_dataset = T5Dataset(val_df, tokenizer, max_len=512)

model = T5ForConditionalGeneration.from_pretrained('t5-base')

training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    evaluation_strategy="steps", # evaluate the model every 'logging_steps'
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./t5_model')


In [ ]:
pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu


In [ ]:
import torch

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS device not found.")
